In [170]:
import torchvision
import pretrainedmodels
import torch
import pretrainedmodels.utils as utils
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from collections import OrderedDict

In [176]:
load_img = utils.LoadImage()

# transformations depending on the model
# rescale, center crop, normalize, and others (ex: ToBGR, ToRange255)
tf_img = utils.TransformImage(model) 

path_img = '/home/dattran/data/xray/00000013_029.png'

input_img = load_img(path_img)
input_tensor = tf_img(input_img)         # 3x400x225 -> 3x299x299 size may differ
input_tensor = input_tensor.unsqueeze(0) # 3x299x299 -> 1x3x299x299
input = torch.autograd.Variable(input_tensor,
    requires_grad=False)

In [178]:
class Resnet(nn.Module):
    def __init__(self, variant):
        super(Resnet, self).__init__()
        assert variant in ['resnet50', 'resnet101', 'resnet152']
        model = pretrainedmodels.__dict__[variant](num_classes=1000, pretrained='imagenet')
        self.features = nn.Sequential(OrderedDict([
            ('conv1', model.conv1),
            ('bn1', model.bn1),
            ('relu', model.relu),
            ('maxpool', model.maxpool),
            ('layer1', model.layer1),
            ('layer2', model.layer2),
            ('layer3', model.layer3),
            ('layer4', model.layer4),
            ('avgpoll', model.avgpool)
        ]))
        num_ftrs = model.last_linear.in_features
        self.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 14),
            nn.Sigmoid()
        )
        
        
    def forward(self, x):
        x = self.features(x) # 1x2048x7x7
        s = x.size()[3] # 7 if input image is 224x224, 16 if input image is 512x512
        x = F.avg_pool2d(x, kernel_size=s, stride=1) # 1x2048x1x1
        x = x.view(x.size(0), -1) # 1x2048
        x = self.classifier(x) # 1x1000
        return x

In [181]:
class DenseNet(nn.Module):
    
    def __init__(self, variant):
        super(DenseNet, self).__init__()
        assert variant in ['densenet121', 'densenet161', 'densenet201']
        model = pretrainedmodels.__dict__[variant](num_classes=1000, pretrained='imagenet')
        self.features = model.features
        num_ftrs = model.last_linear.in_features
        self.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 14),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.features(x) # 1x1024x7x7
        s = x.size()[3] # 7 if input image is 224x224, 16 if input image is 512x512
        x = F.relu(x, inplace=True) # 1x1024x7x7
        x = F.avg_pool2d(x, kernel_size=s, stride=1) # 1x1024x1x1
        x = x.view(x.size(0), -1) # 1x1024
        x = self.classifier(x) # 1x1000
        return x